**Import Libraries**

In [0]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

Scrap data from Wikipedia page into a DataFrame

In [0]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [0]:
# parse data from the html into a beautifulsoup object
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')

In [0]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [0]:
# find the table
soup.find('table').find_all('tr')

In [0]:
# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [0]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})


**Drop cells with a borough that is "Not assigned"**

In [0]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"]

**Group neighborhoods in the same borough**

In [0]:
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"]).agg(lambda x: ', '.join(x)).reset_index()

**For Neighborhood="Not assigned", make the value the same as Borough**

In [0]:
toronto_df_grouped['Neighborhood'] = list(map(lambda x,y: y if x == "Not assigned" else x  ,toronto_df_grouped['Neighborhood'],toronto_df_grouped['Borough']))

**Print the number of rows of the cleaned dataframe**

In [83]:
toronto_df_grouped.shape

(103, 3)